In [5]:
import torch

import os

import copy

import torch.utils.model_zoo.torch_utils

ModuleNotFoundError: No module named 'torch.utils.model_zoo.torch_utils'; 'torch.utils.model_zoo' is not a package

In [2]:
os.listdir('/home/clark/Documents/llama/llama_weights/13B')

ckpt_path_1 = '/home/clark/Documents/llama/llama_weights/13B/consolidated.00.pth'

ckpt_path_2 = '/home/clark/Documents/llama/llama_weights/13B/consolidated.01.pth'

checkpoint_1 = torch.load(ckpt_path_1, map_location="cpu")

checkpoint_2 = torch.load(ckpt_path_2, map_location="cpu")

In [ ]:


model = torch.load("model.pth")

torch.utils.model_zoo.torch_utils.split_parameters(model, 10)

In [4]:
checkpoint_1

{'tok_embeddings.weight': tensor([[ 2.2650e-06,  3.6716e-05,  6.0701e-04,  ...,  2.5868e-04,
           4.5419e-04, -6.6638e-05],
         [ 1.3969e-02, -2.6688e-02,  1.2268e-01,  ..., -2.0996e-02,
           4.0102e-04,  9.7046e-03],
         [-2.0370e-02,  4.9438e-03, -4.5410e-02,  ...,  5.5046e-03,
          -2.5063e-03, -8.8730e-03],
         ...,
         [-2.1210e-02,  3.1067e-02,  7.7553e-03,  ..., -1.6327e-02,
           1.4320e-02,  1.7868e-02],
         [-8.6670e-03, -4.2191e-03, -9.3102e-05,  ..., -7.1001e-04,
          -2.7752e-04, -1.0254e-02],
         [ 1.3451e-02, -2.4139e-02, -1.2703e-02,  ...,  1.6060e-03,
           1.0681e-02, -1.1414e-02]], dtype=torch.float16),
 'norm.weight': tensor([1.1885, 1.1172, 1.2051,  ..., 1.2383, 1.2451, 1.1523],
        dtype=torch.float16),
 'output.weight': tensor([[-0.0088, -0.0117, -0.0034,  ...,  0.0010, -0.0022, -0.0088],
         [ 0.0133, -0.0395, -0.0287,  ..., -0.0233, -0.0375, -0.0101],
         [-0.0024, -0.0190, -0.0096,  ..

In [3]:
weights = checkpoint_1.parameters()
weights_1, weights_2 = torch.split(weights, 2)

AttributeError: 'dict' object has no attribute 'parameters'

In [ ]:
model = torch.load("my_model.pth")

weights = model.parameters()
weights_1, weights_2 = torch.split(weights, 2)

In [3]:
ckpt_path_0 = '/home/clark/Documents/llama/llama_weights/7B/consolidated.00.pth'
checkpoint_0 = torch.load(ckpt_path_0, map_location="cpu")

In [4]:
checkpoint_0['tok_embeddings.weight'].shape

torch.Size([32000, 4096])

In [5]:
checkpoint_0_1 = copy.deepcopy(checkpoint_0)

In [7]:
checkpoint_0_1 = {k: v for k, v in checkpoint_0_1.items() if 'rope.freqs' not in k}
checkpoint_1 = {k: v for k, v in checkpoint_1.items() if 'rope.freqs' not in k}
checkpoint_2 = {k: v for k, v in checkpoint_2.items() if 'rope.freqs' not in k}

In [8]:
for name in checkpoint_0_1:
    print(name)

tok_embeddings.weight
norm.weight
output.weight
layers.0.attention.wq.weight
layers.0.attention.wk.weight
layers.0.attention.wv.weight
layers.0.attention.wo.weight
layers.0.feed_forward.w1.weight
layers.0.feed_forward.w2.weight
layers.0.feed_forward.w3.weight
layers.0.attention_norm.weight
layers.0.ffn_norm.weight
layers.1.attention.wq.weight
layers.1.attention.wk.weight
layers.1.attention.wv.weight
layers.1.attention.wo.weight
layers.1.feed_forward.w1.weight
layers.1.feed_forward.w2.weight
layers.1.feed_forward.w3.weight
layers.1.attention_norm.weight
layers.1.ffn_norm.weight
layers.2.attention.wq.weight
layers.2.attention.wk.weight
layers.2.attention.wv.weight
layers.2.attention.wo.weight
layers.2.feed_forward.w1.weight
layers.2.feed_forward.w2.weight
layers.2.feed_forward.w3.weight
layers.2.attention_norm.weight
layers.2.ffn_norm.weight
layers.3.attention.wq.weight
layers.3.attention.wk.weight
layers.3.attention.wv.weight
layers.3.attention.wo.weight
layers.3.feed_forward.w1.weight


In [8]:
checkpoint_1['tok_embeddings.weight'].shape

torch.Size([32000, 2560])

In [9]:
checkpoint_2['tok_embeddings.weight'].shape

torch.Size([32000, 2560])

In [10]:
for name in checkpoint_0_1:
    print(name)

tok_embeddings.weight
norm.weight
output.weight
layers.0.attention.wq.weight
layers.0.attention.wk.weight
layers.0.attention.wv.weight
layers.0.attention.wo.weight
layers.0.feed_forward.w1.weight
layers.0.feed_forward.w2.weight
layers.0.feed_forward.w3.weight
layers.0.attention_norm.weight
layers.0.ffn_norm.weight
layers.1.attention.wq.weight
layers.1.attention.wk.weight
layers.1.attention.wv.weight
layers.1.attention.wo.weight
layers.1.feed_forward.w1.weight
layers.1.feed_forward.w2.weight
layers.1.feed_forward.w3.weight
layers.1.attention_norm.weight
layers.1.ffn_norm.weight
layers.2.attention.wq.weight
layers.2.attention.wk.weight
layers.2.attention.wv.weight
layers.2.attention.wo.weight
layers.2.feed_forward.w1.weight
layers.2.feed_forward.w2.weight
layers.2.feed_forward.w3.weight
layers.2.attention_norm.weight
layers.2.ffn_norm.weight
layers.3.attention.wq.weight
layers.3.attention.wk.weight
layers.3.attention.wv.weight
layers.3.attention.wo.weight
layers.3.feed_forward.w1.weight


In [11]:
for name, tensor in checkpoint_0_1.items():
    print(name, tensor.shape)

tok_embeddings.weight torch.Size([32000, 4096])
norm.weight torch.Size([4096])
output.weight torch.Size([32000, 4096])
layers.0.attention.wq.weight torch.Size([4096, 4096])
layers.0.attention.wk.weight torch.Size([4096, 4096])
layers.0.attention.wv.weight torch.Size([4096, 4096])
layers.0.attention.wo.weight torch.Size([4096, 4096])
layers.0.feed_forward.w1.weight torch.Size([11008, 4096])
layers.0.feed_forward.w2.weight torch.Size([4096, 11008])
layers.0.feed_forward.w3.weight torch.Size([11008, 4096])
layers.0.attention_norm.weight torch.Size([4096])
layers.0.ffn_norm.weight torch.Size([4096])
layers.1.attention.wq.weight torch.Size([4096, 4096])
layers.1.attention.wk.weight torch.Size([4096, 4096])
layers.1.attention.wv.weight torch.Size([4096, 4096])
layers.1.attention.wo.weight torch.Size([4096, 4096])
layers.1.feed_forward.w1.weight torch.Size([11008, 4096])
layers.1.feed_forward.w2.weight torch.Size([4096, 11008])
layers.1.feed_forward.w3.weight torch.Size([11008, 4096])
layers.

In [12]:
for name, tensor in checkpoint_1.items():
    print(name, tensor.shape)

tok_embeddings.weight torch.Size([32000, 2560])
norm.weight torch.Size([5120])
output.weight torch.Size([16000, 5120])
layers.0.attention.wq.weight torch.Size([2560, 5120])
layers.0.attention.wk.weight torch.Size([2560, 5120])
layers.0.attention.wv.weight torch.Size([2560, 5120])
layers.0.attention.wo.weight torch.Size([5120, 2560])
layers.0.feed_forward.w1.weight torch.Size([6912, 5120])
layers.0.feed_forward.w2.weight torch.Size([5120, 6912])
layers.0.feed_forward.w3.weight torch.Size([6912, 5120])
layers.0.attention_norm.weight torch.Size([5120])
layers.0.ffn_norm.weight torch.Size([5120])
layers.1.attention.wq.weight torch.Size([2560, 5120])
layers.1.attention.wk.weight torch.Size([2560, 5120])
layers.1.attention.wv.weight torch.Size([2560, 5120])
layers.1.attention.wo.weight torch.Size([5120, 2560])
layers.1.feed_forward.w1.weight torch.Size([6912, 5120])
layers.1.feed_forward.w2.weight torch.Size([5120, 6912])
layers.1.feed_forward.w3.weight torch.Size([6912, 5120])
layers.1.atte

In [13]:
for name, tensor in checkpoint_2.items():
    print(name, tensor.shape)

tok_embeddings.weight torch.Size([32000, 2560])
norm.weight torch.Size([5120])
output.weight torch.Size([16000, 5120])
layers.0.attention.wq.weight torch.Size([2560, 5120])
layers.0.attention.wk.weight torch.Size([2560, 5120])
layers.0.attention.wv.weight torch.Size([2560, 5120])
layers.0.attention.wo.weight torch.Size([5120, 2560])
layers.0.feed_forward.w1.weight torch.Size([6912, 5120])
layers.0.feed_forward.w2.weight torch.Size([5120, 6912])
layers.0.feed_forward.w3.weight torch.Size([6912, 5120])
layers.0.attention_norm.weight torch.Size([5120])
layers.0.ffn_norm.weight torch.Size([5120])
layers.1.attention.wq.weight torch.Size([2560, 5120])
layers.1.attention.wk.weight torch.Size([2560, 5120])
layers.1.attention.wv.weight torch.Size([2560, 5120])
layers.1.attention.wo.weight torch.Size([5120, 2560])
layers.1.feed_forward.w1.weight torch.Size([6912, 5120])
layers.1.feed_forward.w2.weight torch.Size([5120, 6912])
layers.1.feed_forward.w3.weight torch.Size([6912, 5120])
layers.1.atte

In [4]:
def nested_dict_compare(d1, d2, path=""):
    for k in d1.keys():
        if k not in d2:
            print(f"Key {path + '/' + k} not found in the second dict")
            return False
        else:
            if isinstance(d1[k], dict):
                if not nested_dict_compare(d1[k], d2[k], path=path + "/" + k):
                    return False
            else:
                if torch.is_tensor(d1[k]) and torch.is_tensor(d2[k]):
                    if not torch.equal(d1[k], d2[k]):
                        print(f"Tensor mismatch at path: {path + '/' + k}")
                        return False
                elif d1[k] != d2[k]:
                    print(f"Value mismatch at path: {path + '/' + k}")
                    return False
    return True

checkpoint_1 = torch.load(ckpt_path_1, map_location="cpu")
checkpoint_2 = torch.load(ckpt_path_2, map_location="cpu")

are_same = nested_dict_compare(checkpoint_1, checkpoint_2)

print("Checkpoints are the same:", are_same)


Tensor mismatch at path: /tok_embeddings.weight
Checkpoints are the same: False
